In [ ]:
import numpy as np
import pandas as pd
import os
import time
import math

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from PIL import Image

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
TARGET_COLUMNS = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']

DEBUG = False

if DEBUG is False:
    BATCH_SIZE = 32
    EPOCHS = 10
    AVERAGING_SIZE = 100
else:
    BATCH_SIZE = 4
    EPOCHS = 2
    AVERAGING_SIZE = 20

ROOT_DIR = '/kaggle/input/ranzcr-clip-catheter-line-classification/train'
OUTPUT_DIR = './'
MODEL_NAME = 'cnn1'
IMG_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load and configure data 

### Load DataFrame with labels of images

In [ ]:
# Load DF with labels 
train_set_df = pd.read_csv('/kaggle/input/ranzcr-clip-catheter-line-classification/train.csv')

if DEBUG is True:
    train_set_df = train_set_df.sample(200)
else:
    train_set_df = train_set_df


train_set_df.shape

### Create custom PyTorch dataset

We do this instead of using ImageFolder as we don't want to reorganize the input folder as it is given from Kaggle already loaded without any nesting, and we have more than one class for each image so we need custom dataset

In [ ]:
class RanzcrClipDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, labels_df, transform=None):
        """
        Args:
            labels_df (string): DataFrame with mapping of images to target
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels_df[TARGET_COLUMNS].values
        self.file_paths = [os.path.join(ROOT_DIR, f"{uid}.jpg") for uid in labels_df["StudyInstanceUID"].values]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        # Read image as PIL
        sample = Image.open(self.file_paths[idx]).convert('RGB')
        
        # Get label vector for this UID
        # Vector of length 11 where there is 1 for each class the image is in, 0 otherwise
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        # Run all given transformations on image
        if self.transform:
            sample = self.transform(sample)

        return (sample, label)

### Transforms

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

### Split test dataset into train and test

In [ ]:
# Train is 75%, Test 25%
train_split_df, val_split_df = train_test_split(train_set_df, test_size=0.25, random_state=42)

train_set = RanzcrClipDataset(labels_df=train_split_df, transform=train_transforms)
val_set = RanzcrClipDataset(labels_df=val_split_df, transform=val_transforms)

print(f'Train size: {len(train_set)}, Validation size: {len(val_set)}')

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)

## Visualize some images

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=classes)


# Training Code

In [ ]:
def valid(net, criterion, val_loader):
    
    y_true = []
    y_pred = []
    y_prob = []
    
    # switch to evaluation mode
    model.eval()
    
    start_time = time.time()
    
    with torch.no_grad():
        loss = 0.0
        for i, (inputs, labels) in enumerate(val_loader, 0):

            inputs = inputs.to(device)
            labels = labels.to(device)
      
            outputs = net(inputs)

            loss += criterion(outputs, labels).item()
            
            softmax = nn.Softmax(dim=1)

            probs = softmax(outputs)
            
            for i in range(len(outputs)):
                y_true.append(labels[i].cpu().detach().numpy())
                y_pred.append(np.round(probs[i].cpu().detach().numpy()))
                y_prob.append(probs[i].cpu().detach().numpy())
        
        y_true = np.vstack(y_true)
        y_pred = np.vstack(y_pred)
        y_prob = np.vstack(y_prob)
        
        del inputs
        del labels
        torch.cuda.empty_cache()
        
        end_time = time.time()
        print(f'EVAL: Elapsed {(end_time - start_time):.4f} Loss: {(loss / len(val_loader)):.4f}') 
        
        return y_true, y_pred, y_prob, (loss / len(val_loader))


def train(net, train_loader, criterion, optimizer, epoch):
    
    train_loss = []
    
    trainloader_size = len(train_loader)
    
    epoch_loss = 0.0    # Cummulative loss for epoch
    running_loss = 0.0  # Loss per averaging size
    
    # switch to train mode
    model.train()
    start = end = time.time()
    
    for i, (inputs, labels) in enumerate(train_loader, 0):

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        epoch_loss += loss.item()

        del inputs
        del labels
        torch.cuda.empty_cache()

        # measure elapsed time
        end = time.time()
        
        if i % AVERAGING_SIZE == 0 or i == (trainloader_size-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Loss: {loss:.4f} '
                  'Time: {elapsed_time:.4f}'
                  .format(epoch+1, i, trainloader_size,
                          loss=(running_loss / AVERAGING_SIZE),
                          elapsed_time=(end - start)))
            train_loss.append(running_loss / AVERAGING_SIZE)
            running_loss = 0.0

    return train_loss, (epoch_loss / len(train_loader))


def run_training(net, train_loader=train_loader, val_loader=val_loader):
    
    train_loss_per_epoch = []
    val_loss = []
    roc_per_epoch = []
    
    net.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-6, amsgrad=False)
    
    for epoch in range(EPOCHS):
        start_time = time.time()

        train_loss, avg_epoch_loss = train(net, train_loader, criterion, optimizer, epoch)
        train_loss_per_epoch.append(avg_epoch_loss)
        
        y_true, y_pred, y_prob, avg_val_loss = valid(net, criterion, val_loader)
        val_loss.append(avg_val_loss)
        
        elapsed = time.time() - start_time
        
        print(f'Epoch {epoch+1} - avg_epoch_loss: {avg_epoch_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        
        try:
            roc = calc_metrics(y_true, y_pred, y_prob)
        except:
            roc = 0
            print("Error calculating metrics")
        
        roc_per_epoch.append(roc)
        
        torch.save({'model': net.state_dict(), 
                    'epoch': epoch,
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': avg_epoch_loss,
                    'train_loss_per_epoch': train_loss_per_epoch,
                    'val_loss': val_loss,
                    'roc_per_epoch': roc_per_epoch},
                    f'{OUTPUT_DIR}{MODEL_NAME}_epoch_{epoch+1}_loss_{avg_val_loss:.4f}_roc_{roc:.4f}.pth')

    
    return train_loss_per_epoch, val_loss, roc_per_epoch


def calc_metrics(y_true, y_pred, y_prob):

    # Calculate accuracy for each label
    acc = [accuracy_score(y_true[:, i], y_pred[:, i]) for i in range(len(TARGET_COLUMNS))]
    print("ACC: ", np.around(acc, decimals=3))
    print("AVG ACC: ", np.around(np.mean(acc),  decimals=3))

    try:
        # Calculate ROC
        roc = [roc_auc_score(y_true[:, i], y_prob[:, i]) for i in range(len(TARGET_COLUMNS))]
        avg_roc = np.around(np.mean(roc),  decimals=3)
        print("ROC: ", np.around(roc, decimals=3))
        print("AVG ROC: ", avg_roc)
    except:
        print("Error calculating roc")
        avg_roc = 0
    
    return avg_roc

# Create Model

In [ ]:
class CNN1(nn.Module):

    def __init__(self):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3)        
        self.conv2 = nn.Conv2d(10, 5, 3)        
        self.conv3 = nn.Conv2d(5, 8, 3)
        self.fc1 = nn.Linear(8 * 61 * 61, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 11)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Train Model!

In [ ]:
model = CNN1()
train_loss, val_loss, roc_per_epoch = run_training(net=model, train_loader=train_loader, val_loader=val_loader)

In [ ]:
plt.plot(train_loss, 'go-', label='train')
plt.plot(val_loss, 'ro-', label='validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(roc_per_epoch, 'go-', label='train')
plt.xlabel('Epoch')
plt.ylabel('Average ROC')
plt.legend()
plt.show()